# Data Cleaning

## Preparation

In [3]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from scipy.stats import mannwhitneyu

In [4]:
#load dataset
df = pd.read_csv("full_data.csv")
#define years
df = df[(df['Year'] >= 1996) & (df['Year'] <= 2022)]

In [5]:
# drop no longer used columns
df = df.drop(columns=['cc', 'ge', 'rl',
       'rq', 'va', 'Fertility rate, total (births per woman)',
       'Individuals using the Internet (% of population)',
       'Life expectancy at birth, total (years)',
       'School enrollment, primary (% gross)',
       'External balance on goods and services (% of GDP)',
       'Inflation, GDP deflator (annual %)',
       'Inflation, consumer prices (annual %)',
       'Total tax and contribution rate (% of profit)',
       'Unemployment, total (% of total labor force) (national estimate)',
       'Gini index']) 

In [6]:
# check remaining columns
df.columns

Index(['Year', 'Country', 'Country Code_x', 'Subregion', 'Homicide index',
       'Homicide Ratio', 'HDI', 'Classification', 'GDP (constant 2015 US$)',
       'Mortality rate, infant (per 1,000 live births)',
       'Unemployment, total (% of total labor force) (modeled ILO estimate)',
       'WGI_composite'],
      dtype='object')

In [7]:
# rename columns to better work with
new_names = ['year', 'country', 'code', 'region', 'homicide_index', 
             'homicide_ratio', 'hdi', 'hdi_class', 'gdp', 'infant_mort',
             'unemployment', 'wgi']
df.columns = new_names
# check
print(df)
df.columns

      year                             country code  \
597   1996                            Anguilla  AIA   
598   1996                             Albania  ALB   
599   1996                             Armenia  ARM   
600   1996                 Antigua and Barbuda  ATG   
601   1996                           Australia  AUS   
...    ...                                 ...  ...   
4194  2022            United States of America  USA   
4195  2022                            Holy See  VAT   
4196  2022    Saint Vincent and the Grenadines  VCT   
4197  2022  Venezuela (Bolivarian Republic of)  VEN   
4198  2022                        South Africa  ZAF   

                               region  homicide_index  homicide_ratio    hdi  \
597   Latin America and the Caribbean             0.0        0.000000    NaN   
598                   Southern Europe           272.0        8.380375  0.644   
599                      Western Asia           114.0        3.476950  0.627   
600   Latin America

Index(['year', 'country', 'code', 'region', 'homicide_index', 'homicide_ratio',
       'hdi', 'hdi_class', 'gdp', 'infant_mort', 'unemployment', 'wgi'],
      dtype='object')

In [8]:
# convert some columns into numeric format
# define explicitly numeric columns
numeric_cols = ['homicide_index',
       'homicide_ratio', 'hdi', 'gdp',
       'infant_mort', 'wgi', 
       'unemployment'
]

# convert each column to numeric and forcing errors to NA
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')


## Deleting high missingness rows

In [10]:
# drop empty countries (defined too differently in each dataset)
countries_to_drop = [
    'United Kingdom (England and Wales)',
    'Iraq (Kurdistan Region)',
    'United Kingdom (Scotland)',
    'United Kingdom (Northern Ireland)',
    'Iraq (Central Iraq)'
]

df = df[~df['country'].isin(countries_to_drop)]

In [11]:
# interpolate missing years for WGI (limit=2, both directions)
df.loc[:, 'wgi'] = df.groupby("country")['wgi'].transform(
    lambda x: x.interpolate(limit=2, limit_direction="both")
)

In [12]:
# check missing per variable
missing_percent = df.isnull().mean().sort_values(ascending=False) #17.68 for many
print("Missing values (%):\n", missing_percent * 100)

print("\n=== Missing Values ===")
missing = df.isna().sum()
print(missing[missing > 0].sort_values(ascending=False))

Missing values (%):
 unemployment      24.059293
hdi               11.602052
hdi_class         11.602052
infant_mort       10.433295
wgi                6.385405
gdp                5.672748
year               0.000000
country            0.000000
code               0.000000
region             0.000000
homicide_index     0.000000
homicide_ratio     0.000000
dtype: float64

=== Missing Values ===
unemployment    844
hdi             407
hdi_class       407
infant_mort     366
wgi             224
gdp             199
dtype: int64


In [13]:
# defining minimum coverage

# define core predictors
core_predictors = ['unemployment', 'hdi', 'infant_mort', 'wgi', 'gdp']

# count missing per country
country_counts = df.groupby('country')['homicide_ratio'].count()

# % of non-missing values for predictors per country
coverage = df.groupby('country')[core_predictors].apply(lambda x: x.notna().mean())

# combine for filtering
summary = coverage.copy()
summary['n_obs'] = country_counts

# at least 8 years of data and at least 60% predictor coverage
min_years = 8
min_avg_coverage = 0.6

filtered_countries = summary[
    (summary['n_obs'] >= min_years) & 
    (summary[core_predictors].mean(axis=1) >= min_avg_coverage)
].index.tolist()

# filter df
df_filtered = df[df['country'].isin(filtered_countries)]

In [14]:
# check change in dataset size
print(df['country'].nunique())
print(df_filtered['country'].nunique())

198
146


In [15]:
# missing values by country
missing_by_country = df_filtered.groupby("country").apply(lambda x: x.isna().mean().mean())
missing_by_country = missing_by_country[missing_by_country > 0.1]  # countries with >10% missing

print("Countries with high missingness (>10%):")
print(missing_by_country.sort_values(ascending=False).to_string())

Countries with high missingness (>10%):
country
Puerto Rico                                       0.250000
Andorra                                           0.166667
China, Hong Kong Special Administrative Region    0.166667
Venezuela (Bolivarian Republic of)                0.166667
Antigua and Barbuda                               0.158333
Saint Kitts and Nevis                             0.145833
Kiribati                                          0.143939
Saint Vincent and the Grenadines                  0.113636


In [16]:
# check current dataset
print("=== Dataset Structure ===")
print(f"Total rows: {len(df_filtered)}")
print(f"Columns ({len(df_filtered.columns)}):\n{df_filtered.columns.tolist()}")
print("\nData types:")
print(df_filtered.dtypes.value_counts())

=== Dataset Structure ===
Total rows: 3057
Columns (12):
['year', 'country', 'code', 'region', 'homicide_index', 'homicide_ratio', 'hdi', 'hdi_class', 'gdp', 'infant_mort', 'unemployment', 'wgi']

Data types:
float64    7
object     4
int64      1
Name: count, dtype: int64


In [17]:
#check remaining missigness
print("\n=== Missing Values ===")
missing = df_filtered.isna().sum()
print(missing[missing > 0].sort_values(ascending=False))


=== Missing Values ===
unemployment    479
hdi              93
hdi_class        93
infant_mort      54
wgi              46
gdp              26
dtype: int64


## Missingness of each variable

In [19]:
missing_by_country = df_filtered.groupby("country")["hdi"].apply(lambda x: x.isna().mean()).sort_values(ascending=False)
print(missing_by_country.head(15))

country
Puerto Rico                         1.000000
Bhutan                              0.583333
Turkmenistan                        0.529412
Antigua and Barbuda                 0.450000
Saint Kitts and Nevis               0.375000
Kiribati                            0.363636
Georgia                             0.222222
Suriname                            0.210526
Uzbekistan                          0.200000
Saint Vincent and the Grenadines    0.181818
Dominica                            0.086957
Grenada                             0.086957
Montenegro                          0.058824
Philippines                         0.000000
Poland                              0.000000
Name: hdi, dtype: float64


In [20]:
missing_by_country = df_filtered.groupby("country")["infant_mort"].apply(lambda x: x.isna().mean()).sort_values(ascending=False)
print(missing_by_country.head(15))

country
Puerto Rico                                       1.0
China, Hong Kong Special Administrative Region    1.0
Afghanistan                                       0.0
Qatar                                             0.0
Paraguay                                          0.0
Peru                                              0.0
Philippines                                       0.0
Poland                                            0.0
Portugal                                          0.0
Republic of Korea                                 0.0
Panama                                            0.0
Republic of Moldova                               0.0
Romania                                           0.0
Russian Federation                                0.0
Rwanda                                            0.0
Name: infant_mort, dtype: float64


In [21]:
missing_by_country = df_filtered.groupby("country")["gdp"].apply(lambda x: x.isna().mean()).sort_values(ascending=False)
print(missing_by_country.head(15)) 

country
Venezuela (Bolivarian Republic of)    1.0000
San Marino                            0.0625
Puerto Rico                           0.0000
Panama                                0.0000
Papua New Guinea                      0.0000
Paraguay                              0.0000
Peru                                  0.0000
Philippines                           0.0000
Poland                                0.0000
Portugal                              0.0000
Qatar                                 0.0000
Oman                                  0.0000
Republic of Korea                     0.0000
Republic of Moldova                   0.0000
Romania                               0.0000
Name: gdp, dtype: float64


In [22]:
missing_by_country = df_filtered.groupby("country")["wgi"].apply(lambda x: x.isna().mean()).sort_values(ascending=False)
print(missing_by_country.head(15))

country
Timor-Leste            1.0
Andorra                1.0
Romania                1.0
Portugal               0.0
Panama                 0.0
Papua New Guinea       0.0
Paraguay               0.0
Peru                   0.0
Philippines            0.0
Poland                 0.0
Afghanistan            0.0
Puerto Rico            0.0
Pakistan               0.0
Republic of Korea      0.0
Republic of Moldova    0.0
Name: wgi, dtype: float64


In [23]:
missing_by_country = df_filtered.groupby("country")["unemployment"].apply(lambda x: x.isna().mean()).sort_values(ascending=False)
print(missing_by_country.head(30))

country
Netherlands (Kingdom of the)                      1.0
Republic of Moldova                               1.0
Slovakia                                          1.0
Dominica                                          1.0
Republic of Korea                                 1.0
Seychelles                                        1.0
Kiribati                                          1.0
Egypt                                             1.0
Tuvalu                                            1.0
Bolivia (Plurinational State of)                  1.0
Türkiye                                           1.0
Grenada                                           1.0
China, Hong Kong Special Administrative Region    1.0
United Republic of Tanzania                       1.0
Saint Lucia                                       1.0
Saint Kitts and Nevis                             1.0
Andorra                                           1.0
Antigua and Barbuda                               1.0
United States of Ame

unemployment: no data for 24 countries

WGI: no data Andorra, Romania, Timor-Leste  

poverty: no data Puerto Rico, Hong Kong

hdi: no data Puerto Rico

gdp: no data Venezuela 


## Imputation

In [26]:
# subregion-level imputation for all countries
predictors = ['hdi', 'infant_mort', 'gdp', 'unemployment', 'wgi']

for var in predictors:
    df_filtered.loc[:, var] = pd.to_numeric(df_filtered[var], errors='coerce')
    df_filtered.loc[:, var] = df_filtered.groupby(['region', 'year'])[var].transform(
        lambda x: x.fillna(x.mean())
    )

In [27]:
# check remaining missingess
na_summary = df_filtered.isna().sum().sort_values(ascending=False)
print(na_summary[na_summary > 0])

hdi_class       93
unemployment    11
hdi              4
dtype: int64


In [28]:
# replace Kiribati missing varieables with average of all Oceania countries
df_filtered.loc[df_filtered['country'] == 'Kiribati', 'unemployment'] = (
    df_filtered[df_filtered['region'].isin(['Melanesia', 'Polynesia', 'Micronesia'])].groupby('year')['unemployment'].transform('mean')
)

df_filtered.loc[df_filtered['country'] == 'Kiribati', 'hdi'] = (
    df_filtered[df_filtered['region'].isin(['Melanesia', 'Polynesia', 'Micronesia'])].groupby('year')['hdi'].transform('mean')
)

In [29]:
# check missingness again
na_summary = df_filtered.isna().sum().sort_values(ascending=False)
print(na_summary[na_summary > 0])

hdi_class    93
dtype: int64


In [30]:
# check if any countries were lost 
print("\nCountry count:", df_filtered['country'].nunique())


Country count: 146


In [31]:
#save as dataset to work with now
df_filtered.to_csv("dissertation_dataset.csv", index=False)
